In [1]:
import time
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime

# -------------------
# 1) Configurable Risk Parameters
# -------------------
RISK_PER_TRADE_PERCENT = 2.0   # Keeping it for reference; no longer used
MAX_DAILY_DD_PERCENT   = 3.0   # If equity drops more than 3% in a day, stop new trades
DEFAULT_LOT_SIZE = 5.0         # Changed to 5.0 lots

SYMBOL = "EURUSD"
TIMEFRAME = mt5.TIMEFRAME_H1
WILLIAMS_R_PERIOD = 14
CHECK_INTERVAL = 3600  # seconds to wait between checks

# -------------------
# 2) Williams %R Function
# -------------------
def calculate_williams_r(df, period=14):
    """
    Calculates Williams %R for the given DataFrame.
    df must have columns: 'High', 'Low', 'Close'.
    """
    if df.empty:
        return df

    highest_high = df['High'].rolling(window=period).max()
    lowest_low = df['Low'].rolling(window=period).min()
    williams_r = -100 * ((highest_high - df['Close']) / (highest_high - lowest_low))
    df['Williams_%R'] = williams_r
    return df

# -------------------
# 3) MT5 Utility Functions
# -------------------
def initialize_mt5(login=None, password=None, server=None):
    """Initialize and log into the MT5 terminal."""
    if not mt5.initialize():
        print(f"initialize() failed. Error code: {mt5.last_error()}")
        quit()
    print("MetaTrader5 package initialized.")

    # Add terminal info checks
    terminal_info = mt5.terminal_info()
    if terminal_info is None:
        print("Failed to get terminal info")
        quit()
    
    print(f"Trade allowed: {terminal_info.trade_allowed}")

    if login and password and server:
        authorized = mt5.login(login=login, password=password, server=server)
        if not authorized:
            print(f"MT5 login failed. Login={login}, Server={server}")
            mt5.shutdown()
            quit()
        else:
            print(f"Logged in to account #{login} on server {server}")

def shutdown_mt5():
    """Shut down the MT5 connection cleanly."""
    mt5.shutdown()
    print("MetaTrader5 connection shut down.")

def place_order(symbol, order_type, target_lot_size, stop_loss=None, take_profit=None, comment="WR"):
    """
    Places multiple small orders until target lot size is reached
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"[Error] Symbol {symbol} not found!")
        return None

    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"[Error] Failed to select symbol {symbol}")
            return None

    step_size = symbol_info.volume_step  # Usually 0.01
    current_total_lots = 0
    successful_orders = []

    print(f"Starting to place orders. Target: {target_lot_size} lots")
    
    while current_total_lots < target_lot_size:
        # Place one small order at a time
        price = symbol_info.ask if order_type == mt5.ORDER_TYPE_BUY else symbol_info.bid
        
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": step_size,  # Always use minimum step size (0.01)
            "type": order_type,
            "price": price,
            "deviation": 10,
            "magic": 123456,
            "comment": f"{comment}_{len(successful_orders)+1}",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK
        }

        if stop_loss is not None:
            request["sl"] = stop_loss
        if take_profit is not None:
            request["tp"] = take_profit

        # Send order
        result = mt5.order_send(request)
        if result is None:
            print(f"order_send() failed, error code: {mt5.last_error()}")
            break
        elif result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"order_send() failed, retcode={result.retcode}")
            print(f"Request: {request}")
            break
        else:
            successful_orders.append(result)
            current_total_lots += step_size
            print(f"Order {len(successful_orders)}: {step_size} lots opened. Total so far: {current_total_lots}")

    # Summary
    print(f"\nOrder placement complete:")
    print(f"Target lots: {target_lot_size}")
    print(f"Actual lots opened: {current_total_lots}")
    print(f"Number of orders: {len(successful_orders)}")

    return successful_orders if successful_orders else None

# -------------------
# 4) Real-Time Strategy Logic
# -------------------
def check_williams_r_signal(symbol="EURUSD", timeframe=mt5.TIMEFRAME_H1, period=14):
    """
    Check for Williams %R signals with added debugging
    """
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 50)
    if rates is None or len(rates) < 2:
        return None

    df = pd.DataFrame(rates)
    df.rename(columns={"time": "Date", "open": "Open", "high": "High",
                      "low": "Low", "close": "Close"}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], unit='s')

    df = calculate_williams_r(df, period=period)
    if 'Williams_%R' not in df.columns:
        return None

    today = df.iloc[-1]
    yesterday = df.iloc[-2]

    # Add debug output
    print(f"Current Williams %R: {today['Williams_%R']}, Previous: {yesterday['Williams_%R']}")

    if yesterday['Williams_%R'] < -80 and today['Williams_%R'] >= -80:
        return "BUY"
    if yesterday['Williams_%R'] > -20 and today['Williams_%R'] <= -20:
        return "SELL"

    return None

def run_williams_r_strategy(symbol="EURUSD", timeframe=mt5.TIMEFRAME_H1):
    """
    Main strategy function with added account info debugging
    """
    # Check account info
    account_info = mt5.account_info()
    if account_info is None:
        print("[Error] Unable to fetch account info")
        return
    print(f"Account balance: {account_info.balance}")
    print(f"Account equity: {account_info.equity}")
    print(f"Account margin: {account_info.margin}")
    print(f"Account free margin: {account_info.margin_free}")

    if has_daily_drawdown_exceeded():
        print("[Risk] Daily drawdown exceeded. Not opening new trades.")
        return

    signal = check_williams_r_signal(symbol, timeframe, period=WILLIAMS_R_PERIOD)
    if not signal:
        return

    # Use 5.0 lot size
    lot = DEFAULT_LOT_SIZE
    print(f"Attempting to open total lot size: {lot}")

    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print("[Error] Symbol info is None")
        return

    ask_price = symbol_info.ask
    bid_price = symbol_info.bid

    if signal == "BUY":
        entry_price = ask_price
        sl_price = entry_price - (symbol_info.point * 100)  # Example SL distance
        tp_price = entry_price + (symbol_info.point * 200)  # Example TP distance

        if lot <= 0:
            print("[Warning] Lot size calculation returned 0 or negative. Trade skipped.")
            return

        place_order(symbol, mt5.ORDER_TYPE_BUY, lot, stop_loss=sl_price, take_profit=tp_price,
                   comment="WR_BUY")

    elif signal == "SELL":
        entry_price = bid_price
        sl_price = entry_price + (symbol_info.point * 100)  # Example SL distance
        tp_price = entry_price - (symbol_info.point * 200)  # Example TP distance

        if lot <= 0:
            print("[Warning] Lot size calculation returned 0 or negative. Trade skipped.")
            return

        place_order(symbol, mt5.ORDER_TYPE_SELL, lot, stop_loss=sl_price, take_profit=tp_price,
                   comment="WR_SELL")

# -------------------
# 5) Daily Drawdown Logic
# -------------------
_daily_start_equity = None
_daily_drawdown_exceeded = False
_current_day = None

def reset_daily_equity_baseline():
    global _daily_start_equity, _daily_drawdown_exceeded
    account_info = mt5.account_info()
    if account_info is not None:
        _daily_start_equity = account_info.equity
        _daily_drawdown_exceeded = False
        print(f"[Daily Reset] Daily baseline equity set to {_daily_start_equity:.2f}")
    else:
        print("[Daily Reset] Could not fetch account info!")

def has_daily_drawdown_exceeded():
    global _daily_drawdown_exceeded, _daily_start_equity

    if _daily_drawdown_exceeded or _daily_start_equity is None:
        return _daily_drawdown_exceeded

    account_info = mt5.account_info()
    if account_info is None:
        return False

    current_equity = account_info.equity
    threshold_equity = _daily_start_equity * (1 - MAX_DAILY_DD_PERCENT / 100)

    if current_equity < threshold_equity:
        print(f"[Drawdown Exceeded] Current equity {current_equity:.2f} < daily threshold {threshold_equity:.2f}")
        _daily_drawdown_exceeded = True
        return True
    return False

# -------------------
# 6) Main Loop
# -------------------
def main():
    # Update with your credentials
    MY_LOGIN = 5032225458
    MY_PASSWORD = "@6IzQoQz"
    MY_SERVER = "MetaQuotes-Demo"  # Update this with your broker's server

    initialize_mt5(login=MY_LOGIN, password=MY_PASSWORD, server=MY_SERVER)

    global _current_day
    _current_day = datetime.now().day
    reset_daily_equity_baseline()

    print(f"Starting Williams %R strategy on {SYMBOL}, timeframe={TIMEFRAME} with risk management...")

    try:
        while True:
            now = datetime.now()
            if now.day != _current_day:
                _current_day = now.day
                reset_daily_equity_baseline()

            run_williams_r_strategy(SYMBOL, TIMEFRAME)

            print(f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] Checked {SYMBOL} for signals. Sleeping {CHECK_INTERVAL}s...")
            time.sleep(CHECK_INTERVAL)

    except KeyboardInterrupt:
        print("Manual shutdown signal received.")
    finally:
        shutdown_mt5()

if __name__ == "__main__":
    main()


MetaTrader5 package initialized.
Trade allowed: True
Logged in to account #5032225458 on server MetaQuotes-Demo
[Daily Reset] Daily baseline equity set to 99970.04
Starting Williams %R strategy on EURUSD, timeframe=16385 with risk management...
Account balance: 99970.0
Account equity: 99970.04
Account margin: 105.09
Account free margin: 99864.95
Current Williams %R: -29.199999999996518, Previous: -8.199999999996162
Attempting to open total lot size: 5.0
Starting to place orders. Target: 5.0 lots
Order 1: 0.01 lots opened. Total so far: 0.01
Order 2: 0.01 lots opened. Total so far: 0.02
Order 3: 0.01 lots opened. Total so far: 0.03
Order 4: 0.01 lots opened. Total so far: 0.04
Order 5: 0.01 lots opened. Total so far: 0.05
Order 6: 0.01 lots opened. Total so far: 0.060000000000000005
Order 7: 0.01 lots opened. Total so far: 0.07
Order 8: 0.01 lots opened. Total so far: 0.08
Order 9: 0.01 lots opened. Total so far: 0.09
Order 10: 0.01 lots opened. Total so far: 0.09999999999999999
Order 1